In [43]:
%pip install -Uq spacy plotly

Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy
from spacy import displacy
import plotly.express as px

In [2]:
nlp = spacy.load("ru_core_news_sm")

In [3]:
files = {
    "./Texts/chudesniy_doktor.txt": "Чудесный Доктор",              # 1897
    "./Texts/nochnaya_smena.txt": "Ночная Смена",                   # 1899
    "./Texts/beliy_pudel.txt": "Белый Пудель",                      # 1903
    "./Texts/poedinok.txt": "Поединок",                             # 1905
    "./Texts/izumrud.txt": "Изумруд",                               # 1907
    "./Texts/granatovy_braslet.txt": "Гранатовый Браслет",          # 1910
    "./Texts/anaphema.txt": "Анафема",                              # 1913
    "./Texts/zvezda_solomona.txt": "Звезда Соломона",               # 1917
    "./Texts/limonnaya_korka.txt": "Лимонная Корка",                # 1920
    "./Texts/sinaya_zvezda.txt": "Синяя Звезда",                    # 1927 
    }
kuprin_sen = {}

for filename, name in files.items():
    with open(filename, "r") as file:
        kuprin_sen.update({name: list(nlp(file.read()).sents)})

In [53]:
kuprin_sen["Изумруд"][100]

Изумруд был уже окончательно высушен, вычищен щетками и вытерт шерстяной рукавицей, когда пришел главный наездник конюшни, англичанин.

In [5]:
for token in kuprin_sen["Изумруд"][100]:
    print(token.text, token.pos_, token.dep_)
displacy.render(kuprin_sen["Изумруд"][100], style="dep", jupyter=True)

Изумруд NOUN nsubj:pass
был AUX aux:pass
уже ADV advmod
окончательно ADV advmod
высушен VERB ROOT
, PUNCT punct
вычищен VERB conj
щетками NOUN obl
и CCONJ cc
вытерт NOUN conj
шерстяной ADJ amod
рукавицей NOUN obl
, PUNCT punct
когда SCONJ mark
пришел VERB advcl
главный ADJ amod
наездник NOUN nsubj
конюшни NOUN nmod
, PUNCT punct
англичанин NOUN conj
. PUNCT punct


In [56]:
class SentenceAnalyser:
    def get_type(self, sentence):
        full_clause = False
        for token in sentence:
            if token.dep_ == "advcl":
                return "Сложно-подчинённое"
            elif token.dep_ == "nsubj" and full_clause == True:
                return "Сложно-сочинённое"
            elif token.dep_ == "nsubj":
                full_clause = True
        return "Простое"

    def get_intonation(self, sentence):         # Почему-то не работает...
        if sentence[-1] == "!":
            return "Восклицательное"
        elif sentence[-1] == "?":
            return "Вопросительное"
        else:
            return "Удтвердительное"

    def get_length(self, sentence):
        return len([_ for _ in sentence if _.dep_ != "punct"])

    def count_similar(self, sentence):
        return len([_ for _ in sentence if _.dep_ == "conj"])

    def analyse(self, sentence):
        return [sentence, self.get_type(sentence), self.get_intonation(sentence), self.get_length(sentence), self.count_similar(sentence)]

In [57]:
sa = SentenceAnalyser()

In [58]:
kuprin_sen["Белый Пудель"][15:20]

[Им сейчас «Гейшу» подавай, «Под двуглавым орлом», из «Продавца птиц» – вальс.,
 Опять-таки трубы эти…,
 Носил я орган к мастеру – и чинить не берется.,
 «Надо, говорит, новые трубы ставить, а лучше всего, говорит, продай ты свою кислую дребедень в музей…,
 вроде как какой-нибудь памятник…»]

In [59]:
kuprin_analysed = {}

for name, sentences in kuprin_sen.items():
    kuprin_analysed.update({name: [sa.analyse(token) for token in sentences]})

In [65]:
kuprin_analysed["Гранатовый Браслет"][123]

[– Мне все равно, я все люблю, – ответила Анна.,
 'Сложно-сочинённое',
 'Удтвердительное',
 8,
 1]

In [11]:
import pandas as pd

In [64]:
summary = pd.DataFrame(columns=["Текст", "СПП", "ССП", "Простые", "Восклицательные", "Вопросительные", "Удтвердительные", "Однородные члены"])
for name, text in kuprin_analysed.items():
    spp = (len([token[1] for token in text if token[1] == "Сложно-подчинённое"])/len(text))*100
    ssp = (len([token[1] for token in text if token[1] == "Сложно-сочинённое"])/len(text))*100
    simple = (len([token[1] for token in text if token[1] == "Простое"])/len(text))*100
    bang = (len([token[2] for token in text if token[2] == "Восклицательное"])/len(text))*100
    qmark = (len([token[2] for token in text if token[2] == "Вопросительное"])/len(text))*100
    fstop = (len([token[2] for token in text if token[2] == "Удтвердительное"])/len(text))*100
    sameness_rate = (sum([token[4] for token in text])/len(text))*100
    summary.loc[len(summary)] = [name, spp, ssp, simple, bang, qmark, fstop, sameness_rate]
    # display(summary)

px.bar(summary, x="Текст", y=["СПП", "ССП", "Простые", "Однородные члены"])

## Выводы:
- В большинстве текстов превалируют простые предложения (во всех текстах их стабильно больше половины) — что характерно жанру реализма
- Интересно, что количество видов предложений примерно пропорционально от текста к тексту
- Заметен паттерн в увеличении количества простых предложений к произведению "Белый Пудель", потом спад; к произведению "Анафема" и "Звезда Соломона", потом опять спад.
- Однордные члены колебались до "Анафемы", потом стабилизировались до достаточно большого числа. Куприн стал более детально расписывать образы. 